<p style='font-size: 24px; text-align: center;'>Foreword</p>

<div style='font-size:16px;'>**This notebook has two big parts:**

<ul>
  <li>**Part 1:** aims to transition all regression tests from one E+ version to the next, and allows you to run each test in both the old and the new version
    <ul>
      <li>These will take quite some time to run (about 1hr to run the tests in the OLD OpenStudio version, transition the IDFs to the new E+ version and run them, and run the tests in the NEW OpenStudio Version, based on almost 200 files currently)</li>
      <li>By default it will just copy over the SQL from the regression test to place in OLD_DIR, but if you want to force rerun the IDF in the old E+ version you can.</li>
      <li>At the end of Part 1, you will have three CSV files, one per version, with the site KBTUs for each test. And you also have an organized tree of VERSION/TEST_NAME/ output directories that have the SQL files we will use for sections 6+.</li>
    </ul>  
  </li>
  <li>**Part 2:** aims to analyze the differences between versions
    <ul>
      <li>Section 3.1 just re-queries all SQL file (or you can reload the three CSV files) to highlight the tests with the biggest site KBTU differences</li>
      <li>Section 3.2 provides a high-level interface that only requires to pass a test name and it will query the relevant SQL files and produce visualization (tables, grouped bar charts, and heatmaps) to analyze where differences may be coming from</li>
    </ul>
    If you have already run Part 1 successfully once, you only need to run Section 1. and you can jump to Part 2 directly.
  </li>
     
</div> 

----

**Note**

* It might be a good idea to monitor your system after each big tasks to ensure you don't have processes that are still hanging. It happened to me for intersection test for eg.

In [ ]:
%matplotlib inline

# Python 2.x / 3.x compatibility
from __future__ import division, print_function

#Import modules
import pandas as pd
import numpy as np
import os
import json
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re

#import csv
import glob as gb

#import pathlib

import datetime
import sqlite3

import shutil
import multiprocessing
import subprocess

import tqdm
import pathlib

from ipywidgets import HTML
from IPython.display import display

import shlex

# from df2gspread import df2gspread as d2g

mpl.rcParams['figure.figsize'] = (16, 9)
pd.options.display.max_rows = 200

# Args

These should match your actual installation.

In [ ]:
IDF_DIR = os.getcwd()
IDF_DIR

In [ ]:
# Path to the OpenStudio-resources/testruns/ directory
# TESTRUNS_DIR = '/home/julien/Software/Others/OpenStudio-resources/testruns/'
TESTRUNS_DIR = os.path.abspath('../testruns')
OS_RES_DIR = os.path.abspath('..')

EPLUS_OLD_VERSION = '8.8.0'
OS_OLD_VERSION = '2.4.2'

EPLUS_NEW_VERSION = '8.9.0'
OS_NEW_VERSION = '2.4.3'

TRANSITION_CLI_DIR = '/home/julien/Software/Others/EnergyPlus-build/Products'
# For some reason this one doesn't work
# TRANSITION_CLI_DIR = '/home/julien/Software/Others/OS-build2/EnergyPlus-8.9.0-1c5ba897d1-Linux-x86_64/EnergyPlus-8-9-0/PreProcess/IDFVersionUpdater/'

# Force a given number of parallel process 
# (defaults to nproc - 2, leaving one physical core free if you have hyperthreading)
N = None
WEATHER_FILE= os.path.join(IDF_DIR, 'USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw')

# Path to EnergyPlus application & idd 
OLD_EPLUS_EXE = '/usr/local/EnergyPlus-8-8-0/energyplus'
NEW_EPLUS_EXE = os.path.join(TRANSITION_CLI_DIR, 'energyplus-8.9.0')
# NEW_EPLUS_EXE = '/home/julien/Software/Others/OS-build2/EnergyPlus-8.9.0-1c5ba897d1-Linux-x86_64/EnergyPlus-8-9-0/energyplus-8.9.0'

# Path to OpenStudio CLIs
# OLD_OS_CLI = '/usr/bin/openstudio-2.4.1'
OLD_OS_CLI = '/home/julien/Software/Others/OS-build/Products/openstudio-2.4.2'
NEW_OS_CLI = '/home/julien/Software/Others/OS-build2/Products/openstudio-2.4.3'

# OLD_IDD_FILE = '/usr/local/EnergyPlus-8-8-0/Energy+.idd'
# NEW_IDD_FILE = os.path.join(TRANSITION_DIR, 'Energy+.idd')

# Put None if you want to run all tests
REGRESSION_TEST_FILTER = '(kiva)|(baseline)'
REGRESSION_TEST_FILTER = None

## Setup

In [ ]:
# For the translation to work,
# you'll have to chdir to the Transition CLI's folder
TRANSITION_CLI = os.path.abspath(os.path.join(TRANSITION_CLI_DIR,
                                             'Transition-V{}-to-V{}'.format(EPLUS_OLD_VERSION.replace('.', '-'),
                                                                EPLUS_NEW_VERSION.replace('.', '-'))))
TRANSITION_CLI

In [ ]:
# Number of parallel processes
if not N:
    N = multiprocessing.cpu_count() - 2
    print("Defaulting number of processes to {}".format(N))

In [ ]:
# Create directories
OLD_OS_DIR = "{o}-{e}".format(e=EPLUS_OLD_VERSION, o=OS_OLD_VERSION)
OLD_OS_DIR = os.path.join(IDF_DIR, OLD_OS_DIR)

NEW_OS_DIR = "{o}-{e}".format(e=EPLUS_NEW_VERSION, o=OS_NEW_VERSION)
NEW_OS_DIR = os.path.join(IDF_DIR, NEW_OS_DIR)

TRANSITION_DIR = "Transition-{e}".format(e=EPLUS_NEW_VERSION)
TRANSITION_DIR = os.path.join(IDF_DIR, TRANSITION_DIR)

for p in [OLD_OS_DIR, NEW_OS_DIR, TRANSITION_DIR]:
    if not os.path.exists(p):
        os.makedirs(p)
        print('Creating directory: {}'.format(p))

In [ ]:
# Create dicts to store all info
OLD_OS_INFO = {'OS_VERSION': OS_OLD_VERSION,
               'EPLUS_VERSION': EPLUS_OLD_VERSION,
               'DIR': OLD_OS_DIR}

NEW_OS_INFO = {'OS_VERSION': OS_NEW_VERSION,
               'EPLUS_VERSION': EPLUS_NEW_VERSION,
               'DIR': NEW_OS_DIR}

TRANSITION_INFO = {'OS_VERSION': 'Transition',
                   'EPLUS_VERSION': EPLUS_NEW_VERSION,
                   'DIR': TRANSITION_DIR}

# PART 1: Transition all regression tests and run them in both E+ versions

TODO/Note: I guess I could just copy the SQL file from the OpenStudio-resources/model_tests.rb too...

## Run in Previous OpenStudio Version based on old E+

This will go in the `TESTRUNS_DIR ` (`OpenStudio-resources/testruns`) directory and find all IDF files and copy them to the `IDF_DIR` directory (typically the directory in which this notebook resides)

<span style="font-size: 18px; color: red;">It goes without saying: you need to have already run all simulation tests with the last OpenStudio version that is based on the old E+ version before running this section.</span>

### Run the tests in the old version

You can also just do that manually... but if you do, please delete the testruns/ folder beforehand.

In [ ]:
# Delete all testruns to ensure we don't end up grabbing the idf and sql from another version
if os.path.exists(TESTRUNS_DIR):
    shutil.rmtree(TESTRUNS_DIR)

In [ ]:
# Pass 'CUSTOMTAG=' if you don't want a tag, or 'CUSTOMTAG=sha' for the build sha,
# or any custom string such as 'CUSTOMTAG=Ubuntu_run1'
CUSTOMTAG=''

if REGRESSION_TEST_FILTER is None:
    filt = ''
else:
    filt = "-n /{}/".format(REGRESSION_TEST_FILTER)

command = "env CUSTOMTAG={c} {cli} {m} {filt}".format(c=CUSTOMTAG,
                                                      m=os.path.join(OS_RES_DIR,
                                                                     'model_tests.rb'),
                                                      cli=OLD_OS_CLI,
                                                      filt=filt)
print(command)
c_args = shlex.split(command)

# Run it
process = subprocess.Popen(c_args, shell=False,
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
# wait for the process to terminate
#out, err = process.communicate()
#errcode = process.returncode
for line in iter(process.stdout.readline, b''):
    l = line.rstrip().decode()
    if 'extensions are not built' in l:
        continue
    print(l)
process.stdout.close()
process.wait()

## Copy the Previous IDFs

These end up directly in IDF_DIR. They will get copied to the `OLD_OS_DIR` during the Transition

In [ ]:
# Cleanup directory
all_files = gb.glob(os.path.join(IDF_DIR, '*.idf'))
all_files += gb.glob(os.path.join(IDF_DIR, '*.idfnew'))
all_files += gb.glob(os.path.join(IDF_DIR, '*.idfold'))

for f in all_files:
    os.remove(f)

In [ ]:
found_idfs = []
for f in gb.iglob(os.path.join(TESTRUNS_DIR, '**/*/in.idf')):
    f2 = os.path.relpath(f, TESTRUNS_DIR)
    
    test_name = os.path.split(os.path.split(f2)[0])[0]
    #print(test_name)
    dst_path = os.path.join(IDF_DIR, "{}.idf".format(test_name))
    shutil.copyfile(f, dst_path)
    found_idfs.append(test_name)
found_idfs = set(found_idfs)

## Copy all existing SQL files

In [ ]:
found_sqls = []

for f in gb.iglob(os.path.join(TESTRUNS_DIR, '**/*/*.sql')):
    f2 = os.path.relpath(f, TESTRUNS_DIR)
    
    test_name = os.path.split(os.path.split(f2)[0])[0]
    # print(test_name)
    dst_folder = os.path.join(OLD_OS_DIR, test_name)
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    dst_path = os.path.join(dst_folder, "eplusout.sql")
    # print(dst_path)
    shutil.copyfile(f, dst_path)
    found_sqls.append(test_name)
found_sqls = set(found_sqls)

In [ ]:
len(found_idfs), len(found_sqls)

In [ ]:
found_idfs - found_sqls

## Transition all files

In [ ]:
def translate_file(path):
    """
    Runs the file throught the transition utility and save in the right folder
    Will move the ori file to the subdirectory OLD_DIR (eg `./8.8.0/`)
    and the transitionned one to NEW_DIR (eg: `./8.9.0/`)
    """
    
    eplus_file, ext = os.path.splitext(os.path.split(path)[1])
    
    process = subprocess.Popen([TRANSITION_CLI, path],
                               shell=False,
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE)
    # wait for the process to terminate
    out, err = process.communicate()
    errcode = process.returncode
    if errcode == 0:        
        # Move the resulting IDF into the new dir
        new_file = os.path.join(IDF_DIR, "{f}.idfnew".format(f=eplus_file))
        new_dest = os.path.join(TRANSITION_DIR, "{f}.idf".format(f=eplus_file))
        shutil.move(new_file, new_dest)
        
        # Move the old version into its directory
        old_file = os.path.join(IDF_DIR, "{f}.idfold".format(f=eplus_file))
        old_dest = os.path.join(OLD_OS_DIR, "{f}.idf".format(f=eplus_file))
        shutil.move(old_file, old_dest)
        
        # Delete original file
        ori_file = os.path.join(IDF_DIR, "{f}.idf".format(f=eplus_file))
        os.remove(ori_file)
        
        # print('Done for {}.idf - {}'.format(eplus_file, path))
    else:
        print("Error for {}".format(path))
        print(out)
        print(err)

In [ ]:
files = [os.path.abspath(file) for file in gb.glob(os.path.join(IDF_DIR, '*.idf'))]
len(files)

In [ ]:
files[:N]

In [ ]:
TRANSITION_CLI_DIR

In [ ]:
# You must cd to the Transition CLI's folder for it to work
os.chdir(TRANSITION_CLI_DIR)

# Takes about 10minutes on my machine with 12 threads allocated
pool = multiprocessing.Pool(processes=N)

desc = '<h3>Translation from {} to {}</h3>'.format(EPLUS_OLD_VERSION,
                                                   EPLUS_NEW_VERSION)
label = HTML(desc)
display(label)
for _ in tqdm.tqdm_notebook(pool.imap_unordered(translate_file, files), total=len(files)):
    pass

os.chdir(IDF_DIR)

In [ ]:
# At this point, you shouldn't have any .idf files in the IDF_DIR directory
# If you do, means that the transition failed
all_files = gb.glob(os.path.join(IDF_DIR, '*.idf'))
all_files += gb.glob(os.path.join(IDF_DIR, '*.idfnew'))
all_files += gb.glob(os.path.join(IDF_DIR, '*.idfold'))

all_files

## Run Simulation in E+

In [ ]:
GJ_TO_KBTU= 947.8171203133173

SQL_QUERY_TOTAL_SITE_KBTU = "SELECT Value FROM tabulardatawithstrings WHERE \
                              ReportName='AnnualBuildingUtilityPerformanceSummary' AND \
                              ReportForString='Entire Facility' AND \
                              TableName='Site and Source Energy' AND \
                              RowName='Total Site Energy' AND \
                              ColumnName='Total Energy' AND \
                              Units='GJ'"

SQL_QUERY_SIM_INFO = 'SELECT EnergyPlusVersion FROM Simulations'

VERSION_REGEX = re.compile(r'Version (?P<Major>\d+)\.(?P<Minor>\d+)\.'
                                   '(?P<Patch>\d+)-(?P<SHA>\w+),\s+'
                                   'YMD=(?P<datestring>[0-9\.: ]+)')


# Remove all files in the output directory except these
KEEP_EXT = ['.err', '.sql']

In [ ]:
def parse_sql_version_and_sitekbtu(output_directory):
    """
    This function grabs the EnergyPlusVersion and the total site energy
    from the SQL file.
    
    Args:
    -----
    * output_directory (str): the path were the SQL should be.
        eg: `./8.8.0/absorption_chillers.osm_8.8.0/`
    
    Returns:
    ---------
    * pd.Series that has the version with SHA and site kbtu
        (or None if it didn't run), which name is the test_name
        (gotten from the name of the output_directory)
    """
    sql_files = gb.glob(os.path.join(output_directory, "*.sql"))

    version_with_sha = None
    site_kbtu = None

    if len(sql_files) == 1:
        sql_path = sql_files[0]
        abs_sql_path = os.path.abspath(sql_path)
        sql_uri = '{}?mode=ro'.format(pathlib.Path(abs_sql_path).as_uri())
        with sqlite3.connect(sql_uri, uri=True) as con:
                cursor = con.cursor()
                r = cursor.execute(SQL_QUERY_SIM_INFO).fetchone()
                if r:
                    simulation_info = r[0]
                    m = VERSION_REGEX.search(simulation_info)
                    if m:
                        gpdict = m.groupdict()
                        version_with_sha = "{}.{}.{}-{}".format(gpdict['Major'],
                                                                     gpdict['Minor'],
                                                                     gpdict['Patch'],
                                                                     gpdict['SHA'])
                else:
                    msg = ("Cannot find the EnergyPlusVersion in the SQL file. "
                           "For:\n{}".format(output_directory))
                    #raise ValueError(msg)
                    print(msg)

                # Get Site kBTU
                r = cursor.execute(SQL_QUERY_TOTAL_SITE_KBTU).fetchone()
                if r:
                    site_gj = float(r[0])
                    site_kbtu = site_gj * GJ_TO_KBTU
                    msg = ("Cannot find the Total Site Energy in the SQL file. "
                           "For:\n{}".format(output_directory))
    return pd.Series([version_with_sha, site_kbtu],
                     index=['E+', 'SiteKBTU'],
                     name = os.path.split(output_directory)[1])

In [ ]:
def run_OLD_eplus_sim(eplus_file):
    """
    Runs the simulation with OLD_EPLUS_EXE and calls parse_sql
    """
    base, file_with_ext = os.path.split(os.path.abspath(eplus_file))
    output_directory = os.path.join(base, os.path.splitext(file_with_ext)[0])
    # If directory exists, delete it
    if os.path.exists(output_directory):
        shutil.rmtree(output_directory)
    # Recreate it
    os.makedirs(output_directory)
    
    process = subprocess.Popen([OLD_EPLUS_EXE,
                                # '-i', OLD_IDD_FILE
                                '-w', WEATHER_FILE,
                                '-d', output_directory,
                               eplus_file],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               universal_newlines=True, 
                               shell=False)
    
    # wait for the process to terminate
    out, err = process.communicate()
    errcode = process.returncode
    if errcode == 0:
        # Clean up output directory
        [os.remove(x) for x in gb.glob(os.path.join(output_directory, '*'))
         if os.path.splitext(x)[1] not in KEEP_EXT]
        return parse_sql_version_and_sitekbtu(output_directory)
    else:
        print("ERROR: {}".format(eplus_file))
        print(out)
        print(err)
        return pd.Series([None, None],
                     index=['E+', 'SiteKBTU'],
                     name = os.path.split(output_directory)[1])
    
def run_NEW_eplus_sim(eplus_file):
    """
    Runs the simulation with NEW_EPLUS_EXE and calls parse_sql
    """
    base, file_with_ext = os.path.split(os.path.abspath(eplus_file))
    output_directory = os.path.join(base, os.path.splitext(file_with_ext)[0])
    # If directory exists, delete it
    if os.path.exists(output_directory):
        shutil.rmtree(output_directory)
    # Recreate it
    os.makedirs(output_directory)
    
    process = subprocess.Popen([NEW_EPLUS_EXE,
                                # '-i', OLD_IDD_FILE
                                '-w', WEATHER_FILE,
                                '-d', output_directory,
                               eplus_file],
                               stdout=subprocess.PIPE,
                               stderr=subprocess.PIPE,
                               universal_newlines=True, 
                               shell=False)
    
    # wait for the process to terminate
    out, err = process.communicate()
    errcode = process.returncode
    if errcode == 0:
        # Clean up output directory
        [os.remove(x) for x in gb.glob(os.path.join(output_directory, '*'))
         if os.path.splitext(x)[1] not in KEEP_EXT]
        return parse_sql_version_and_sitekbtu(output_directory)
    else:
        print("ERROR: {}".format(eplus_file))
        # print(out)
        # print(err)
        return pd.Series([None, None],
                     index=['E+', 'SiteKBTU'],
                     name = os.path.split(output_directory)[1])

### Run all 8.8.0 files or just load SQL

#### Rerun in old E+

This should take about 10-15 minutes depending on your machine.

It is currently disabled (as RawNBConvert) because we should have already copied the needed SQL files from the old OpenStudio version that is based on the old EnergyPlus Version. Switch this cell to "Code" if you do want to rerun with your old installed E+ version

#### Just parse copied SQLs

In [ ]:
old_results = pd.concat([parse_sql_version_and_sitekbtu(os.path.join(OLD_OS_DIR, x)) 
                         for x in os.listdir(OLD_OS_DIR)
                         if os.path.isdir(os.path.join(OLD_OS_DIR, x))],
                        axis=1).T

old_results['OS'] = OS_OLD_VERSION

old_results.to_csv(os.path.join(IDF_DIR, 'kbtus_{o}-{e}.csv'.format(e=EPLUS_OLD_VERSION,
                                                                    o=OS_OLD_VERSION)))

In [ ]:
old_results.head()

### Run all 8.9.0 files

This should take about 10-15 minutes depending on your machine.

In [ ]:
files = gb.glob(os.path.join(TRANSITION_DIR, '*.idf'))
files[:N]

In [ ]:
# About 15-20minutes with 12 threads for all tests
files = gb.glob(os.path.join(TRANSITION_DIR, '*.idf'))

pool = multiprocessing.Pool(processes=N)

desc = '<h3>Running Transitioned files in E+ {}</h3>'.format(EPLUS_NEW_VERSION)
label = HTML(desc)
display(label)
all_results = []
for result in tqdm.tqdm_notebook(pool.imap_unordered(run_NEW_eplus_sim, files), total=len(files)):
    all_results.append(result)

In [ ]:
# Concat in dataframe and save to CSV
transitioned_results = pd.concat(all_results, axis=1).T
transitioned_results['OS'] = 'Transition'
transitioned_results.to_csv(os.path.join(IDF_DIR, 'kbtus_Transition-{e}.csv'.format(e=EPLUS_NEW_VERSION)))

In [ ]:
transitioned_results.head()

In [ ]:
len(old_results.index), len(transitioned_results.index)

In [ ]:
set(old_results.index) - set(transitioned_results.index)

## Run Simulation in new OpenStudio based on new EnergyPlus

### Run New OS Version regression tests

In [ ]:
# Delete all testruns to ensure we don't end up grabbing the idf and sql from another version
# model_tests.rb only cleans out testruns/testXXX directories for tests we do request
# So if you use a regression test filter, you could have left overs
if os.path.exists(TESTRUNS_DIR):
    shutil.rmtree(TESTRUNS_DIR)

In [ ]:
# Pass 'CUSTOMTAG=' if you don't want a tag, or 'CUSTOMTAG=sha' for the build sha,
# or any custom string such as 'CUSTOMTAG=Ubuntu_run1'
CUSTOMTAG=''

if REGRESSION_TEST_FILTER is None:
    filt = ''
else:
    filt = "-n /{}/".format(REGRESSION_TEST_FILTER)

command = "env CUSTOMTAG={c} {cli} {m} {filt}".format(c=CUSTOMTAG,
                                                      m=os.path.join(OS_RES_DIR,
                                                                     'model_tests.rb'),
                                                      cli=NEW_OS_CLI,
                                                      filt=filt)
print(command)
c_args = shlex.split(command)
c_args

In [ ]:
process = subprocess.Popen(c_args, shell=False,
                           stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
# wait for the process to terminate
#out, err = process.communicate()
#errcode = process.returncode
lines = []
for line in iter(process.stdout.readline, b''):
    print(line.rstrip().decode())
    lines.append(line)
process.stdout.close()
process.wait()
    
#os.chdir(IDF_DIR)

### Copy IDF

Copy to the `NEW_OS_DIR` directly

In [ ]:
found_idfs = []
for f in gb.iglob(os.path.join(TESTRUNS_DIR, '**/*/in.idf')):
    f2 = os.path.relpath(f, TESTRUNS_DIR)
    
    test_name = os.path.split(os.path.split(f2)[0])[0]
    #print(test_name)
    dst_path = os.path.join(NEW_OS_DIR, "{}.idf".format(test_name))
    shutil.copyfile(f, dst_path)
    found_idfs.append(test_name)
found_idfs = set(found_idfs)

### Copy SQL

In [ ]:
found_sqls = []

for f in gb.iglob(os.path.join(TESTRUNS_DIR, '**/*/*.sql')):
    f2 = os.path.relpath(f, TESTRUNS_DIR)
    
    test_name = os.path.split(os.path.split(f2)[0])[0]
    # print(test_name)
    dst_folder = os.path.join(NEW_OS_DIR, test_name)
    if not os.path.exists(dst_folder):
        os.makedirs(dst_folder)
    dst_path = os.path.join(dst_folder, "eplusout.sql")
    # print(dst_path)
    shutil.copyfile(f, dst_path)
    found_sqls.append(test_name)
found_sqls = set(found_sqls)

In [ ]:
len(found_idfs), len(found_sqls)

In [ ]:
found_idfs - found_sqls

### Parse new SQLs

In [ ]:
new_results = pd.concat([parse_sql_version_and_sitekbtu(os.path.join(NEW_OS_DIR, x)) 
                         for x in os.listdir(NEW_OS_DIR)
                         if os.path.isdir(os.path.join(NEW_OS_DIR, x))],
                        axis=1).T

new_results['OS'] = OS_NEW_VERSION

new_results.to_csv(os.path.join(IDF_DIR, 'kbtus_{o}-{e}.csv'.format(e=EPLUS_NEW_VERSION,
                                                                    o=OS_NEW_VERSION)))

In [ ]:
new_results.head()

# PART 2: Analyzing differences

## Reparse SQLs

In [ ]:
# Alternatively, we could just reparse the SQLs...

old_results = pd.concat([parse_sql_version_and_sitekbtu(os.path.join(OLD_OS_DIR, x)) 
                         for x in next(os.walk(OLD_OS_DIR))[1]],
                        axis=1).T
old_results['OS'] = OS_OLD_VERSION


transitioned_results = pd.concat([parse_sql_version_and_sitekbtu(os.path.join(TRANSITION_DIR, x)) 
                        for x in next(os.walk(TRANSITION_DIR))[1]],
                        axis=1).T
transitioned_results['OS'] = 'Transition'


new_results = pd.concat([parse_sql_version_and_sitekbtu(os.path.join(NEW_OS_DIR, x)) 
                         for x in next(os.walk(NEW_OS_DIR))[1]],
                        axis=1).T
new_results['OS'] = OS_NEW_VERSION


In [ ]:
len(old_results.index), len(transitioned_results.index), len(new_results)

In [ ]:
set(old_results.index) - set(transitioned_results.index)

In [ ]:
set(old_results.index) - set(new_results.index)

In [ ]:
old_results.to_csv(os.path.join(IDF_DIR,
                                'kbtus_{o}-{e}.csv'.format(e=EPLUS_OLD_VERSION,
                                                           o=OS_OLD_VERSION)))

transitioned_results.to_csv(os.path.join(IDF_DIR,
                                         'kbtus_Transition-{e}.csv'.format(e=EPLUS_NEW_VERSION)))

new_results.to_csv(os.path.join(IDF_DIR,
                                'kbtus_{o}-{e}.csv'.format(e=EPLUS_NEW_VERSION,
                                                           o=OS_NEW_VERSION)))

## Reload site kbtu csvs

In [ ]:
os.chdir(IDF_DIR)

## Concat frames

In [ ]:
df = pd.concat([old_results.dropna(how='all'),
                transitioned_results.dropna(how='all'),
                new_results.dropna(how='all')])

df = df.set_index(['E+', 'OS'], append=True).unstack([1,2])['SiteKBTU']

In [ ]:
# Problems
df[df.isnull().any(axis=1)]

In [ ]:
# Throwing out these problems for further analysis (they need to be investigated on the site)
df = df.dropna(how='any', axis=0)

## First look at where we have deviations

In [ ]:
pct_threshold = 0.0001
print("Setting % diff threshold to {:.3%}".format(pct_threshold))

### Deviations in Transition and/or new OpenStudio

In [ ]:
df_diff = df.pct_change(axis=1).iloc[:, 1:].dropna()
df_diff = df_diff[(df_diff >= pct_threshold).any(axis=1)]
#df_diff = df_diff.sort_values(by=df_diff.columns[-1],
#                              ascending=True)
# Sort by max absolute diff
df_diff = df_diff.loc[df_diff.abs().max(axis=1).sort_values(ascending=True).index]

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, nrows=1, sharex=True, sharey=True,
                               figsize=(16, len(df_diff)/2))
df_diff.iloc[:,0].plot(kind='barh', ax=ax0)
df_diff.iloc[:,1].plot(kind='barh', ax=ax1)

vals = ax0.get_xticks()
ax0.set_xticklabels(['{:3.2f}%'.format(x*100) for x in vals])

ax0.set_title("% difference between {}\nand {}".format(df.columns[0], df.columns[1]))
ax0.set_xlabel("% difference")

ax1.set_title("% difference between {}\nand {}".format(df.columns[1], df.columns[2]))
ax1.set_xlabel("% difference")

sns.despine()

plt.show()

### Deviations from Transition to new OS only

In [ ]:
new_os_diff = df_diff[df_diff.iloc[:,-1] != 0]
new_os_diff.style.format('{:.5%}')

In [ ]:
fig, (ax0, ax1) = plt.subplots(ncols=2, nrows=1, sharex=True, sharey=True,
                               figsize=(16, len(new_os_diff)/2))
new_os_diff.iloc[:,0].plot(kind='barh', ax=ax0)
new_os_diff.iloc[:,1].plot(kind='barh', ax=ax1)

vals = ax0.get_xticks()
ax0.set_xticklabels(['{:3.2f}%'.format(x*100) for x in vals])

ax0.set_title("% difference between {}\nand {}".format(df.columns[0], df.columns[1]))
ax0.set_xlabel("% difference")

ax1.set_title("% difference between {}\nand {}".format(df.columns[1], df.columns[2]))
ax1.set_xlabel("% difference")

sns.despine()

plt.show()

## Grouped bar chart of differences compared to the old OpenStudio

**How to read this chart**:

The percentage differences are calculated compared to the Old OpenStudio results for both the transitioned results and the new OpenStudio results.

**What you need to pay special attention to is when you don't have the same difference between the Transition to Old OS and the New OS to old OS** (meaning the difference is not E+'s fault, but OpenStudio's).

In [ ]:
df2 = df.copy()
df2.columns = df2.columns.droplevel(0)

df_diff_from_old_os = df2.iloc[:,1:].subtract(df2.iloc[:,0], axis=0).divide(df2.iloc[:,0], axis=0)

# Keep only over threshold
df_diff_from_old_os = df_diff_from_old_os[(df_diff_from_old_os >= pct_threshold).any(axis=1)]

# Sort by max absolute diff
df_diff_from_old_os = df_diff_from_old_os.loc[df_diff_from_old_os.abs().max(axis=1).sort_values(ascending=True).index]

fig, ax = plt.subplots(figsize=(16, len(df_diff_from_old_os)/2))
df_diff_from_old_os.plot(kind='barh', stacked=False, ax=ax)

vals = ax.get_xticks()
ax.set_xticklabels(['{:3.2f}%'.format(x*100) for x in vals])

ax.set_title("% difference compared to  {}".format(df.columns[0]))
ax.set_xlabel("% difference")

sns.despine()

plt.show()

# Sort the other way round for the table
(df_diff_from_old_os
    .loc[df_diff_from_old_os.abs().max(axis=1)
                            .sort_values(ascending=False).index]
    .style.format('{:.2%}'))

#html = df_diff_from_old_os.style.format('{:.2%}').render()
#display(HTML(html))

## Stacked bar chart of differences

In [ ]:
def find_ymax_bylabel(label, ax, y_is_top=False):
    """
    Given a label, find the y for that bar, and the max x based on two stacked
    bars
    """
    for i, x in enumerate(ax.get_yticklabels()):
        if x.get_text() == label:
            # y = x.get_position()[1]
            
            # Find the max x between the two rects
            rect1 =  ax.patches[i]
            rect_1_xmax = rect1.get_x() + rect1.get_width()
            
            if y_is_top:
                y = rect1.get_y() + rect1.get_height()
            else:
                y = rect1.get_y() + rect1.get_height() / 2.0
                y = x.get_position()[1]
            
            rect2 = ax.patches[int(i+(len(ax.patches) / 2))]
            rect_2_xmax = rect2.get_x() + rect2.get_width()
            
            return y, max(rect_1_xmax, rect_2_xmax)
    return None, None

def plot_stacked_bar_difference_compared_to_base(toplot):
    fig, ax = plt.subplots(figsize=(16, len(toplot)/2))

    # Total % change from old os to new os
    s_tot_change = toplot.sum(axis=1)

    toplot.plot(kind='barh', stacked=True, ax=ax)

    vals = ax.get_xticks()
    ax.set_xticklabels(['{:3.2f}%'.format(x*100) for x in vals])

    ax.set_title("% difference compared to  {}".format(df.columns[0]))
    ax.set_xlabel("% difference")

    # for i, rect in enumerate(ax.patches):
    #     label = ax.get_yticklabels()[int(i % (len(ax.patches) / 2))].get_text()
    #     tot_change = s_tot_change[label]
    #     ax.annotate("{}-{}-{:.3%}".format(i, label, tot_change), 
    #                 xy=(rect.get_x()+rect.get_width(), rect.get_y()))

    # Need to draw first otherwise we can't get the position
    ax.figure.canvas.draw()



    # Add TOTAL % change (sum of both)           
    for label, val in s_tot_change.items():
        ymid, xmax = find_ymax_bylabel(label=label, ax=ax, y_is_top=False)
        ytop, xmax = find_ymax_bylabel(label=label, ax=ax, y_is_top=True)
        if xmax is not None:
            ax.annotate("{:.3%}".format(val),
                        xy=(val, ytop), xycoords='data',
                        ha='center', va='bottom', color='k', fontsize=8,
                        xytext=(0, 4), textcoords='offset points') 
            ax.plot(val, ytop, marker='v', c='#494949', alpha=1, zorder=3)

    # Custom annotations
    label = 'baseline_sys07.rb'
    y, xmax = find_ymax_bylabel(label=label, ax=ax, y_is_top=False)
    if y is not None:
        ax.annotate("Slight OpenStudio deviation here", xy=(xmax, y), xycoords='data',
                    ha='left', va='center',
                    xytext=(20, 0), textcoords='offset points',
                    arrowprops=dict(arrowstyle="->",
                                connectionstyle="arc3"))

    label = 'centralheatpumpsystem.rb'
    y, xmax = find_ymax_bylabel(label=label, ax=ax, y_is_top=False)
    if y is not None:
        ax.annotate("This ruby test is unstable, period", xy=(xmax, y), xycoords='data',
                    ha='left', va='center',
                    xytext=(20, 0), textcoords='offset points',
                    arrowprops=dict(arrowstyle="->",
                                connectionstyle="arc3")) 

    sns.despine()



    plt.show()

**How to read this chart**:

The percentage differences are calculated compared to the Old OpenStudio results for both the transitioned results and the new OpenStudio results. Then I do

    % diff New version = % diff New version - % diff Transition

and plot that as a stacked bar chart.
The goal is to more clearly see the differences that are due to OpenStudio by removing the differences due to the new E+.

**What you need to pay special attention to is when you see % differences for the new OS.**

**A cursor along with the total % difference between Old OS and New OS is also plotted**.
Please see the below example to get a better sense of how the graph is constructed

In [ ]:
df_diff_from_old_example = pd.DataFrame([[0.003, 0.0005],
                                         [-0.0004, +0.0003],
                                         [0.0002, 0.0004]],
                                        index=['test1.rb', 'test2.osm', 'test3.rb'],
                                        columns=['Transition', '2.4.3']
                                       )
plot_stacked_bar_difference_compared_to_base(df_diff_from_old_example)

**Let's do the actual plotting now:**

In [ ]:
df_diff_from_old_os_as_pct = df_diff_from_old_os.copy()
df_diff_from_old_os_as_pct.loc[:,OS_NEW_VERSION] = df_diff_from_old_os_as_pct.loc[:,OS_NEW_VERSION] - df_diff_from_old_os_as_pct.loc[:,'Transition']

plot_stacked_bar_difference_compared_to_base(df_diff_from_old_os_as_pct)

## Inspect biggest differences

In [ ]:
def get_annual_energy_by_fuel_and_enduse(sql_path):
    """
    Queries SQL file and returns the ABUPS' End Uses table

    Parameters
    ----------
    sql_path (str): path to the sql file

    Returns
    -------
    df_annual: pd.DataFrame
        Annual End Use table
        index = 'EndUse'
        columns = ['FuelType','Units']
    """

    abs_sql_path = os.path.abspath(sql_path)
    sql_uri = '{}?mode=ro'.format(pathlib.Path(abs_sql_path).as_uri())
    
    # RowName = '#{end_use}'
    # ColumnName='#{fuel_type}'
    annual_end_use_query = """SELECT RowName, ColumnName, Units, Value
        FROM TabularDataWithStrings
        WHERE ReportName='AnnualBuildingUtilityPerformanceSummary'
        AND ReportForString='Entire Facility'
        AND TableName='End Uses'
    """

    with sqlite3.connect(sql_uri, uri=True) as con:
        df_annual = pd.read_sql(annual_end_use_query, con=con)

    # Convert Value to Float
    df_annual['Value'] = pd.to_numeric(df_annual['Value'])

    df_annual = df_annual.set_index(['RowName',
                                     'ColumnName',
                                     'Units'])['Value'].unstack([1, 2])
    df_annual.index.name = 'EndUse'
    df_annual.columns.names = ['FuelType', 'Units']

    end_use_order = ['Heating', 'Cooling',
                     'Interior Lighting', 'Exterior Lighting',
                     'Interior Equipment', 'Exterior Equipment',
                     'Fans', 'Pumps', 'Heat Rejection', 'Humidification',
                     'Heat Recovery', 'Water Systems',
                     'Refrigeration', 'Generators']

    col_order = ['Electricity', 'Natural Gas', 'Additional Fuel',
                 'District Cooling', 'District Heating', 'Water']

    df_annual = df_annual[col_order].loc[end_use_order]

    return df_annual

In [ ]:
def find_sql_file(test, version_info):
    """
    Find the sql file given a test name and the version_info dict
    
    Args:
    ------
    * test (str): the test name
    * version_info (dict): should have at least one 'DIR' key with the path
    to the directory
    """
    search_path = os.path.join(version_info['DIR'], test, "*.sql")

    sql_files = gb.glob(search_path)

    if len(sql_files) == 0:
        return None
    elif len(sql_files) > 1:
        print("Found more than one sql file for {t} in "
              "{p}".format(t=test, p=search_path))
    return sql_files[0]

In [ ]:
def parse_single_end_use(test, version_info):
    """
    Helper to load the end use by fuel
    
    Args:
    ------
    * test (str): the test name
    * version_info (dict): should have at least one 'DIR' key with the path
    to the directory
    """
    sql_path = find_sql_file(test, version_info=version_info)
    if sql_path is None:
        print("Cannot find the sql file for test '{}' and version "
              "{}".format(test, version_info['OS_VERSION']))
        return None

    end_use = get_annual_energy_by_fuel_and_enduse(sql_path)
    end_use.columns = pd.MultiIndex.from_tuples([(version_info['OS_VERSION'],) + x for x in end_use.columns],
                                                names = ['Version'] + end_use.columns.names)
    return end_use

In [ ]:
def parse_before_after_enduse(test, old_os=True, transition=True, new_os=True):
    """
    Given a test name, will parse both the old and the new SQL file to return
    a table that has, for both versions, the end use by fuel values
    
    Args:
    ------
    * test (str): the name of the test. eg 'foundation_kiva.osm'
    * old_os, transition, new_os (bool): whether to include these versions
    Note that it relies on the respective global dictionaries
    OLD_OS_INFO, TRANSITION_INFO, NEW_OS_INFO
    
    Returns:
    --------
    * df_all_end_use (pd.DataFrame): a multiindex dataframe of end uses by fuel
        index: ['EndUse'] ('Heating', 'Cooling', etc)
        columns = ['Version', 'FuelType', 'Units']

    """
    concat_list = []
    
    if (old_os + transition + new_os) < 2:
        print("You should request at least 2 versions to compare them...")
        return False
    
    if old_os:
        concat_list.append(parse_single_end_use(test, OLD_OS_INFO))
    if transition:
        concat_list.append(parse_single_end_use(test, TRANSITION_INFO))
    if new_os:
        concat_list.append(parse_single_end_use(test, NEW_OS_INFO))

        
    df_all_end_use = pd.concat([x for x in concat_list if x is not None],
                               axis=1)

    return df_all_end_use

In [ ]:
def plot_end_use_diff(df_all_end_use, test,
                      add_legend=True, fontsize=None,
                      outer_i=None, fig=None):
    """
    Plots the difference in end use by fuel between the new and old versions
    Will have subplots by units (water versus energy), a subplot is only shown
    if there is consumption in the said end use (eg if no Water, there will 
    be only one subplot)
    Displays a grouped bar chart by end use, and annotates % difference in the
    given end use
    
    Args:
    ------
    * both_end_use (pd.DataFrame): dataframe from `parse_before_after_enduse`
    
    * outer_i and fig: if you want to customize the layout yourself. 
        Pass None otherwise (default)
        
    * old_info, new_info (dict): dict with information such as 'DIR', 
    'EPLUS_VERSION' and 'OS_VERSION'
    
    Returns:
    --------
    * None, displays a plot

    """
    
    if fontsize is None:
        fontsize = 10
    
    diff = (df_all_end_use.groupby(level=['Version', 'Units'], axis=1).sum()
                          .replace(0, np.nan)
                          .dropna(how='all', axis=0)
                          .dropna(how='all', axis=1))

    # Reorder properly
    diff = diff[[x for x in [OLD_OS_INFO['OS_VERSION'],
                 TRANSITION_INFO['OS_VERSION'],
                 NEW_OS_INFO['OS_VERSION']] if x in diff.columns]]
    
    grouped = diff.groupby(level='Units', axis=1)

    ncols = min(len(grouped), 2)
    nrows = int(np.ceil(grouped.ngroups/ncols))

    # If you don't supply outer_i, we take care of everything
    if outer_i is None:
        fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(8*ncols,5*nrows))
        if not isinstance(axes, np.ndarray):
            axes = np.array([axes])
    else:
        inner = mpl.gridspec.GridSpecFromSubplotSpec(nrows, ncols,
                    subplot_spec=outer_i, wspace=0.1, hspace=0.1)
        
        #np.array([["{},{}".format(x,y) for y in range(ncols)] for x in range(nrows)])
        axes = np.array([plt.Subplot(fig, inner[j]) for j in range(nrows*ncols)])
        [fig.add_subplot(ax) for ax in axes]

    # Plot each subplot
    first_legend = add_legend
    for (key, ax) in zip(grouped.groups.keys(), axes.flatten()):
        gp = grouped.get_group(key)
        gp.columns= gp.columns.droplevel('Units')
        gp.index.name = ''
        
        # Sort by max absolute difference
        gp = gp.loc[gp.apply(lambda row: max(row) - min(row), axis=1)
                      .sort_values(ascending=False).index]
        gp.plot(kind='bar', ax=ax)
        if key == 'GJ':
            title = "Energy (GJ)"
        elif key == 'm3':
            title = "Water (m3)"
        else:
            # shouldn't happen
            title = key
            
        # Add labels with fontsize
        ax.set_title(title, fontsize=fontsize+2)
        ax.set_ylabel(key, fontsize=fontsize)
        # Set tick size
        for tick in ax.xaxis.get_major_ticks():
            tick.label.set_fontsize(fontsize)
            tick.label.set_rotation(45)
        for tick in ax.yaxis.get_major_ticks():
            tick.label.set_fontsize(fontsize)
            
        # Add % difference if any
        for i, x in enumerate(ax.get_xticklabels()):
            # Return 'Heating', 'Cooling', etc
            idx = x.get_text()

            # Loop on each successive versions
            for k in range(len(gp.columns)-1):
                v_old = gp.loc[idx].iloc[k]
                v_new = gp.loc[idx].iloc[k+1]
                if abs(v_new - v_old) > 0:
                    pct = (v_new - v_old) / v_old

                    if v_old > v_new:
                        # Base on the first bar
                        rect = ax.patches[i+k*len(ax.patches) // len(gp.columns)]
                        # Offset needed to be at mid between both bars
                        x_offset = rect.get_width()
                    else:
                        # Based on second bar
                        rect = ax.patches[i+(k+1)*len(ax.patches) // len(gp.columns)]
                        x_offset = 0

                    ax.annotate(xy=(rect.get_x() + x_offset, rect.get_height()+0.05),
                                xytext=(15*(2*k-1),20), textcoords='offset points',
                                s="{:.2%}".format(pct),
                                ha='center', va='bottom',
                                fontweight='normal',
                                fontsize=fontsize-2, color='k',
                                arrowprops=dict(arrowstyle="->",
                                connectionstyle="arc3"))
        
        # Display legend or not
        if not first_legend:
            ax.legend().set_visible(False)
        else:
            ax.legend()
        first_legend = False
            
    sns.despine()
    
    # Title
    title = "End Use for {}".format(test)
    # fig.suptitle(title)
    axes[0].annotate(title,
                     xy=(0.5*ncols, 1.0), xycoords='axes fraction',
                     xytext=(0, 20), textcoords='offset points',
                     va='bottom', ha='center',
                     fontsize=fontsize+4, fontweight='bold',
                     )
    if outer_i is None:
        plt.show()

In [ ]:
def table_absolute_difference(df_all_end_use, is_incremental=True):
    """
    Computes the absolute difference in (GJ/m3).
    If is_incremental, compares from one version to the next
        eg: returns [Transition - 2.4.2] and '2.4.3 - Transition')
    if false, compares to the oldest one
        eg: returns [Transition - 2.4.2] and '2.4.3 - 2.4.2')

    Args:
    ------
    * df_all_end_use (pd.DataFrame): dataframe from parse_before_after_enduse
    * is_incremental (bool): compare each version to the previous version
        or to the oldest one

    Returns:
    --------
    * abs_diff (pd.DataFrame)
    * html_abs = HTML object
    
    """
    # Sort in the right order
    cols_in_order = [x for x in [OLD_OS_INFO['OS_VERSION'],
                                 TRANSITION_INFO['OS_VERSION'],
                                 NEW_OS_INFO['OS_VERSION']] 
                     if x in df_all_end_use.columns]
        
    abs_diff = df_all_end_use.copy()
    for i, col in enumerate(cols_in_order[1:]):
        if is_incremental:
            k = i
        else:
            k = 0
        abs_diff[col] = df_all_end_use[col] - df_all_end_use[cols_in_order[k]]
    abs_diff = abs_diff[cols_in_order[1:]]
    
    abs_diff = (abs_diff.replace(0, np.nan)
                        .dropna(how='all', axis=0)
                        .dropna(how='all', axis=1))
    
    if is_incremental:
        ann = "<strong>Comparing from one version to the next</strong>"
    else:
        ann = "<strong>Comparing each version to {}</strong>".format(cols_in_order[0])
    if abs_diff.empty:
        # print("There are ZERO absolute differences for {}".format(test))
        html = HTML('<p>{}</p>\n<p style="font-size: 18px; text-align: center">'
                    'There are <strong>ZERO</strong> absolute differences '
                    'for {}</p>'.format(ann, test))
    else:
        html = (abs_diff.style.set_table_styles(styles)
                 .set_caption("Absolute diff for {}\n{}".format(test, ann))
                 .format(lambda x: "{:.0f}".format(x) if not np.isnan(x) else '-'))
        # display(html)
    return abs_diff, html

def table_percent_difference_by_end_use_and_fuel(df_all_end_use,
                                                 is_incremental=True):
    """
    Computes the percentage difference in between the old and the new for each
    end use and fuel.
    
    eg: Heating Electricity % is calculated as
        (heating-electricity-kbtu-new) - (heating-electricity-kbtu-old)
        / (heating-electricity-kbtu-old)
    
    Args:
    ------
    * df_all_end_use (pd.DataFrame): dataframe from parse_before_after_enduse
    
    Returns:
    --------
    * pct_diff (pd.DataFrame)
    * html_diff = HTML object
    """
    # Sort in the right order
    cols_in_order = [x for x in [OLD_OS_INFO['OS_VERSION'],
                                 TRANSITION_INFO['OS_VERSION'],
                                 NEW_OS_INFO['OS_VERSION']] 
                     if x in df_all_end_use.columns]
        
    pct_diff = df_all_end_use.copy()
    for i, col in enumerate(cols_in_order[1:]):
        if is_incremental:
            k = i
        else:
            k = 0
        pct_diff[col] = (df_all_end_use[col] - df_all_end_use[cols_in_order[k]]) / df_all_end_use[cols_in_order[k]]

    pct_diff = pct_diff[cols_in_order[1:]]
    
    pct_diff = (pct_diff.replace(0, np.nan)
                        .dropna(how='all', axis=0)
                        .dropna(how='all', axis=1))
    
    if is_incremental:
        ann = "<strong>Comparing from one version to the next</strong>"
    else:
        ann = "<strong>Comparing each version to {}</strong>".format(cols_in_order[0])
    
    if pct_diff.empty:
        # print("There are ZERO percentage differences for {}".format(test))
        html = HTML('<p>{}</p><p style="font-size: 18px; text-align: center">'
                    'There are <strong>ZERO</strong> percentage differences '
                    'for {}</p>'.format(ann, test))
    else:
        html = (pct_diff.style.set_table_styles(styles)
                 .set_caption("Relative individual % diff for each end use and"
                              " fuel for '{}'\n{}".format(test, ann))
                 .format(lambda x: "{:.2%}".format(x) if not np.isnan(x) else '-'))
        # display(html)
    return pct_diff, html

def table_percent_difference_of_total(df_all_end_use, is_incremental=True):
    """
    Computes the percentage difference in between the old and the new for each
    type Water or GJ.
    
    eg: Heating % is calculated as
        sum(heating-GJ-each-fuel-new) - sum(heating-GJ-each-fueld-old)
        / sum(all_GJ)
    
    Args:
    ------
    * df_all_end_use (pd.DataFrame): dataframe from parse_before_after_enduse
    * is_incremental (bool): compare each version to the previous version
        or to the oldest one
        
    Returns:
    --------
    * percentage_of_total (pd.DataFrame)
    * html_tot = HTML object
  
    """
    
    # Sort in the right order
    cols_in_order = [x for x in [OLD_OS_INFO['OS_VERSION'],
                                 TRANSITION_INFO['OS_VERSION'],
                                 NEW_OS_INFO['OS_VERSION']] 
                     if x in df_all_end_use.columns]

    concat_dict = {}
    for i, col in enumerate(cols_in_order[1:]):
        if is_incremental:
            k = i
        else:
            k = 0
        sum_old = (df_all_end_use[cols_in_order[k]]
                   .groupby(level='Units', axis=1).sum().sum())
        abs_diff_end_use = ((df_all_end_use[col] 
                            - df_all_end_use[cols_in_order[k]])
                            .stack(0)
                            .groupby(level='EndUse').sum())

        percentage_of_total = (abs_diff_end_use / sum_old)
        d = {'GJ': 'Energy', 'm3': 'Water'}
        percentage_of_total.columns = pd.MultiIndex.from_tuples([(d[x], x) 
                                                                 for x in percentage_of_total.columns],
                                                                names=['Type', 'Units'])
        concat_dict[col] = percentage_of_total
    
    percentage_of_total = pd.concat(concat_dict, axis=1)[cols_in_order[1:]]
    # Drop end uses where we have nothing
    percentage_of_total = (percentage_of_total.reindex(df_all_end_use.index)
                                               .replace(0, np.nan)
                                               .dropna(how='all', axis=0)
                           )

    # drop Fuel Type (units) where none of the versions have a change
    sum_by_type =  percentage_of_total.groupby(level='Units', axis=1).sum().sum()
    percentage_of_total.loc[:,
                            (percentage_of_total.columns
                             .get_level_values('Units')
                             .isin(sum_by_type.index[sum_by_type != 0]))]
    if is_incremental:
        ann = "<strong>Comparing from one version to the next</strong>"
    else:
        ann = "<strong>Comparing each version to {}</strong>".format(cols_in_order[0])
    
    if percentage_of_total.empty:
        # print("There are ZERO percentage differences for {}".format(test))
        html = HTML('<p>{}</p><p style="font-size: 18px; text-align: center">'
                    'There are <strong>ZERO</strong> percentage differences '
                    'for {}</p>'.format(ann, test))
    else:
        html = (percentage_of_total.style.set_table_styles(styles)
                 .set_caption("% diff of total GJ/m3 {}\n{}".format(test, ann))
                 .format(lambda x: "{:.2%}".format(x) if not np.isnan(x) else '-'))
        # display(html)
    return percentage_of_total, html

In [ ]:
def report_for_test(test, is_incremental=True,
                    old_os=True, transition=True, new_os=True,
                    plot_if_no_diff=True, add_legend=True,
                    display_tables=False,
                    plot_heatmap=False, heatmap_as_pct_of_total=False,
                    outer_i=None, fig=None):
    """
    High level method to investigate differences
    
    Args:
    -----
    * test (str): test name, eg 'centralheatpumpsystem.rb
    * is_incremental (bool): compare each version to the previous version
        or to the oldest one

    * old_os, transition, new_os (bool): whether to include these versions
    Note that it relies on the respective global dictionaries
    OLD_OS_INFO, TRANSITION_INFO, NEW_OS_INFO
    * display_tables (bool): if true, shows `table_absolute_difference`
        and `table_percent_difference_table`
    * plot_if_no_diff (bool): if there are no difference, whether to show 
        `plot_end_use_diff` anyways or not
    * gs (matplotlib GridSpec): Pass one if you want to organize the figures
        in a given layout, otherwise a new plot is created
    
    Returns:
    --------
    * fig (matplotlib.figure)
    
    Can display requested things on the fly too
    
    """
    has_diff = False
    
    df_all_end_use = parse_before_after_enduse(test,
                                               old_os=old_os,
                                               transition=transition,
                                               new_os=new_os)
    abs_diff, html_abs = table_absolute_difference(df_all_end_use, is_incremental)
    # display(html_abs)
    if not abs_diff.empty:
        has_diff = True
        if display_tables:
            pct_diff, html_pct = table_percent_difference_by_end_use_and_fuel(df_all_end_use, is_incremental)
            percentage_of_total, html_tot = table_percent_difference_of_total(df_all_end_use, is_incremental)

            display(HTML("""
            <div style='display: grid; grid-template-columns: 1fr 1fr 1fr; grid-column-gap: 10px;'>
                <div style='align-self: center; width: '> {html_abs} </div>
                <div style='align-self: center;'> {html_pct} </div>
                <div style='align-self: center;'> {html_tot} </div>
            </div>""".format(html_abs=html_abs.render(),
                             html_pct=html_pct.render(),
                             html_tot=html_tot.render())))
        
    else:
        display(html_abs)
    
    if (has_diff | plot_if_no_diff):
        plot_end_use_diff(df_all_end_use=df_all_end_use, test=test,
                          outer_i=outer_i, fig=fig,
                          add_legend=add_legend)
        
    if has_diff & plot_heatmap:
        if heatmap_as_pct_of_total:
            plot_heatmap_pct_diff(test=test, pct_diff=percentage_of_total,
                                  is_incremental=is_incremental,
                                  as_pct_of_total=True)
        else:
            plot_heatmap_pct_diff(test=test, pct_diff=pct_diff,
                                  is_incremental=is_incremental,
                                  as_pct_of_total=False)
        
def plot_heatmap_pct_diff(test, is_incremental=False,
                          pct_diff=None, df_all_end_use=None,
                          ax=None, figsize=None, short_title=False,
                          as_pct_of_total=False, vmax=None):
    """
    Plots a heatmap of percentage difference. It will show the xlabels
    as "Fuel" only if one unit, if more than one it's "Fuel-Units"
    
    Args:
    -----
    * test (str): test name
    * pct_diff (pd.DataFrame): from `table_percent_difference_table(both_end_use)`
        If not supplied, it is recomputed
        * as_pct_of_total (bool): if True, calls `table_percent_difference_of_total`
        otherwise calls `table_percent_difference_by_end_use_and_fuel`
    * df_all_end_use (pd.DataFrame): from `parse_before_after_enduse(test)`
        If pct_diff is not supplied, it uses this dataframe to recompute pct_diff
        If also not supplied, it is recomputed
    * ax (matplotlib.axes._subplots.AxesSubplot): The axis on which to plot,
        pass None to create a new figure
    * figsize (tuple of int): force a given figure size
        pass None to autocalculate
    * short_title (bool): display only the test name or also with versions
    * vmax (float, typically between 0 and 1): the maximum for the colorbar
        if None, defaults to 0.25 (25%) is as_pct_of_total is False, and
        0.05 (5%) if as_pct_of_total is True
    Returns:
    --------
    None, plots the heatmap
    """
    if vmax is None:
        if as_pct_of_total:
            # Colorbar goes from 0 (yellow) to 5% (red)
            vmax = 0.05
        else:
            vmax = 0.25
    
    # Modularity in arguments, compute only what's needed
    if pct_diff is None:
        if df_all_end_use is None:
            df_all_end_use = parse_before_after_enduse(test)
        if as_pct_of_total:
            pct_diff, _ = table_percent_difference_of_total(df_all_end_use=df_all_end_use,
                                                            is_incremental=is_incremental)
        else:
            pct_diff, _ = table_percent_difference_by_end_use_and_fuel(df_all_end_use=df_all_end_use,
                                                                       is_incremental=is_incremental)

    show_plot = False
    if ax is None:
        show_plot = True
        if figsize is None:
            w = min(pct_diff.shape[0], 16)
            h = pct_diff.shape[0] * w / (3*pct_diff.shape[1])
        else:
            w = figsize[0]
            h = figsize[1]
        fig, ax = plt.subplots(figsize=(w, h))

    fmt = lambda x,pos: '{:.0%}'.format(x)

    toplot = pct_diff.copy()
    if len(toplot.columns.get_level_values('Units').unique()) == 1:
        toplot.columns = toplot.columns.droplevel('Units')

    sns.heatmap(toplot.abs(),
                ax=ax, cmap='YlOrRd',
                vmin=0, vmax=vmax,
                cbar_kws={'format': mpl.ticker.FuncFormatter(fmt)},
                annot=toplot, fmt='.2%')
    if short_title:
        title = test
    else:
        if as_pct_of_total:
            title = ("Percent difference of total GJ/m3 by End Use for test "
                     "'{}'".format(test))
        else:
            title = ("Relative Individual % diff for each End Use / Fuel for test "
                     "'{}'".format(test))
        if is_incremental:
            title += '\nComparing from one version to the next'
        else:
            title += '\nComparing each to the oldest version ({})'.format(OLD_OS_INFO['OS_VERSION'])
    ax.set_title(title)
    if show_plot:
        plt.show()

In [ ]:
def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="th", props=[("font-size", "110%"),
                               ("text-align", "center")]),
    dict(selector="td", props=[("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "bottom"),
                                    ("text-align", "center")])
]

In [ ]:
# Make graph centered on page
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

### Single test

In [ ]:
# Has differences, but no water
test = 'foundation_kiva.osm'
report_for_test(test, is_incremental=False,
                old_os=True, transition=True, new_os=True,
                plot_if_no_diff=True, display_tables=True,
                plot_heatmap=True, heatmap_as_pct_of_total=False)

In [ ]:
 # Zero difference, but has water
test = 'water_heaters.rb'
report_for_test(test, is_incremental=False,
                old_os=True, transition=True, new_os=True,
                plot_if_no_diff=True, display_tables=True, 
                plot_heatmap=True, heatmap_as_pct_of_total=False)

In [ ]:
# has differences in both energy and water
test = 'plenums.rb'
# Let's plot non incremental
report_for_test(test, is_incremental=False,
                old_os=True, transition=True, new_os=True,
                plot_if_no_diff=True, display_tables=True,
                plot_heatmap=True, heatmap_as_pct_of_total=True)

#### Inspect in more detail

In [ ]:
test = 'baseline_sys07.rb'
df_all_end_use = parse_before_after_enduse(test,
                                           old_os=True, transition=True,
                                           new_os=True)

In [ ]:
# Show absolute values for only fuel/end use that have a value
df_all_end_use.replace(0, np.nan).dropna(how='all', axis=0).dropna(how='all', axis=1)

In [ ]:
pct_diff, html = table_percent_difference_by_end_use_and_fuel(df_all_end_use)
display(html)

In [ ]:
# If you don't supply pct_diff, it will be computed again
# if also you don't supply both_end_use, it will be computed again
# Do notice the is_incremental keyword again...
plot_heatmap_pct_diff(test, pct_diff=None,
                      is_incremental=True,
                      df_all_end_use=None,
                      figsize=(16,9), short_title=False,
                      # Switch as_pct_of_total to see the difference
                      as_pct_of_total=True, vmax=None)

### Multiple Tests - one per row

In [ ]:
# We only really care about differences between Transition and new OS
s_diff = df_diff.iloc[:, -1]
s_diff.name

In [ ]:
report_for_largest_n = 6

add_legend=True
for test in s_diff.nlargest(report_for_largest_n).index:
    report_for_test(test, plot_if_no_diff=True, display_tables=False, 
                    add_legend=True)
    add_legend=False

### Multiple Tests - in a grid

In [ ]:
s_diff.nlargest(10)

In [ ]:
report_for_largest_n = 10

outer_ncols = 2
outer_nrows = int(np.ceil(report_for_largest_n/outer_ncols))

fig = plt.figure(figsize=(16, 4*outer_nrows))
outer = mpl.gridspec.GridSpec(outer_nrows, outer_ncols, wspace=0.1, hspace=0.5)

add_legend = True
for i, test in enumerate(s_diff.nlargest(report_for_largest_n).index):
    df_all_end_use = parse_before_after_enduse(test)
    plot_end_use_diff(df_all_end_use=df_all_end_use, test=test, 
                      outer_i=outer[i], 
                      fig=fig, fontsize=7, add_legend=add_legend)
    add_legend = False

# fig.tight_layout()
plt.show()

### Multiple Tests - heatmaps

In [ ]:
report_for_largest_n = 10

ncols = 2
nrows = int(np.ceil(report_for_largest_n/outer_ncols))

fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 4*outer_nrows))

add_legend = True
for test, ax in zip(s_diff.nlargest(report_for_largest_n).index, axes.flatten()):
    # Switch as_pct_of_total, force vmax (max of colorbar) if you want
    plot_heatmap_pct_diff(test, ax=ax,
                          is_incremental=True,
                          short_title=True,
                          as_pct_of_total=True, vmax=None)
    

fig.tight_layout()
plt.show()

# Strip output cell

In [ ]:
# Local import
import os
import glob as gb
from bs4 import BeautifulSoup

#filelist = gb.glob('*.html')
filelist = ['Analysis_from_2.4.2(8.8.0)_to_2.4.3(8.9.0).html']

for s_path in filelist:
    print(s_path)
    
    save_path = "{}_stripped.html".format(os.path.splitext(s_path)[0])
    print("Deleting input cells and warnings")
    with open(s_path,"r+") as htmlDoc:
        soup = BeautifulSoup(htmlDoc, "lxml")
        # Get input divs
        tg = soup.find_all(attrs={"class": "input"})
        # Add input stderr (warnings and errors)
        tg += soup.find_all(attrs={"class": "output_stderr"})
        # Replace with nothing
        for i in range(len(tg)):
            tg[i].replace_with("")

    # Prettify
    html = soup.prettify("utf-8")
    #exportpath = os.path.splitext(htmlpath)[0]+'-noinput'+os.path.splitext(htmlpath)[1]

    # Write
    with open(save_path, "wb") as file:
        file.write(html)

In [ ]:
s_path = filelist[0]
with open(s_path,"r+") as htmlDoc:
    soup = BeautifulSoup(htmlDoc, "lxml")

In [ ]:
soup